# Weather Data Analysis (Google Colab Version)
Analysis and visualization of weather observations and forecasts from EGLC (London City Airport)

**Data Sources:**
- Weather Underground (actual observations & forecasts)
- ThingSpeak API (Thames River water temperature)

## Step 1: Mount Google Drive (for Colab)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Step 2: Set the path to your database
**Update the path below to match where your database is stored in Google Drive**

In [ ]:
# IMPORTANT: Update this path to match your Google Drive folder structure
# Example: '/content/drive/MyDrive/Weather Predictor/data/weather_data.db'

import os

db_path = '/content/drive/MyDrive/Weather Predictor/data/weather_data.db'

# Check if file exists
if os.path.exists(db_path):
    print(f"✓ Database found at: {db_path}")
    print(f"Database size: {os.path.getsize(db_path) / 1024:.2f} KB")
else:
    print(f"✗ Database not found at: {db_path}")
    print(f"\nCurrent directory: {os.getcwd()}")
    print(f"\nPlease update the db_path variable above to point to your database file.")
    print(f"\nYou can browse your Google Drive files to find the correct path.")

## Step 3: Import Libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print("✓ All libraries imported successfully")

## Step 4: Connect to Database

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect(db_path)
print(f"✓ Connected to database")

## Step 5: Load Data from Database

In [ ]:
# Load and display ALL actual weather observations
query_obs = """
SELECT 
    id,
    observation_timestamp,
    location,
    temperature_f,
    dew_point_f,
    humidity_pct,
    wind_speed_mph,
    wind_direction,
    wind_gust_mph,
    pressure_in,
    precip_amount_in,
    condition,
    water_temp_0_35m_c,
    water_temp_2m_c,
    water_temp_7m_c,
    water_temp_entry_id,
    scrape_timestamp
FROM weather_observations
ORDER BY observation_timestamp
"""

df_actual = pd.read_sql_query(query_obs, conn)

# Convert timestamp columns to datetime (handles both space and 'T' separator formats)
df_actual['observation_timestamp'] = pd.to_datetime(df_actual['observation_timestamp'], format='ISO8601', errors='coerce')
df_actual['scrape_timestamp'] = pd.to_datetime(df_actual['scrape_timestamp'], format='ISO8601', errors='coerce')

# Remove rows with invalid timestamps (NaT)
df_actual = df_actual.dropna(subset=['observation_timestamp'])

print("="*80)
print("ACTUAL WEATHER OBSERVATIONS")
print("="*80)
print(f"Total observations: {len(df_actual)}")
if len(df_actual) > 0:
    print(f"Date range: {df_actual['observation_timestamp'].min()} to {df_actual['observation_timestamp'].max()}")
    print(f"Location: {df_actual['location'].unique()}")
print("="*80)
print("\nAll actual observations:")
display(df_actual)

In [ ]:
# Load and display ALL weather forecasts
query_forecast = """
SELECT 
    id,
    forecast_timestamp,
    scrape_timestamp,
    location,
    temperature_f,
    feels_like_f,
    dew_point_f,
    humidity_pct,
    wind_speed_mph,
    wind_direction,
    pressure_in,
    precip_chance_pct,
    precip_amount_in,
    cloud_cover_pct,
    condition
FROM weather_forecasts
ORDER BY forecast_timestamp, scrape_timestamp
"""

df_forecast = pd.read_sql_query(query_forecast, conn)

# Convert timestamp columns to datetime (handles both space and 'T' separator formats)
df_forecast['forecast_timestamp'] = pd.to_datetime(df_forecast['forecast_timestamp'], format='ISO8601', errors='coerce')
df_forecast['scrape_timestamp'] = pd.to_datetime(df_forecast['scrape_timestamp'], format='ISO8601', errors='coerce')

# Remove rows with invalid timestamps (NaT)
df_forecast = df_forecast.dropna(subset=['forecast_timestamp'])

print("="*80)
print("WEATHER FORECASTS")
print("="*80)
print(f"Total forecasts: {len(df_forecast)}")
if len(df_forecast) > 0:
    print(f"Forecast date range: {df_forecast['forecast_timestamp'].min()} to {df_forecast['forecast_timestamp'].max()}")
    print(f"Scrape date range: {df_forecast['scrape_timestamp'].min()} to {df_forecast['scrape_timestamp'].max()}")
    print(f"Location: {df_forecast['location'].unique()}")
print("="*80)
print("\nAll forecasts:")
display(df_forecast)

In [ ]:
# Align forecasted data with actual data using timestamp proximity (within 30 minutes)
import pandas as pd
from datetime import timedelta

# Lists to track matching and missing data
matched_rows = []
missing_forecast_positions = []
missing_actual_positions = []

print("="*80)
print("TIMESTAMP ALIGNMENT (30-minute tolerance)")
print("="*80)

# If we have both actual and forecast data, perform alignment
if len(df_actual) > 0 and len(df_forecast) > 0:
    # For each actual observation, find the closest forecast
    for idx, actual_row in df_actual.iterrows():
        actual_time = actual_row['observation_timestamp']

        # Calculate time differences between this actual observation and all forecasts
        time_diffs = abs(df_forecast['forecast_timestamp'] - actual_time)
        
        # Skip if no forecasts available or all are NaT
        if len(time_diffs) == 0 or time_diffs.isna().all():
            missing_forecast_positions.append({
                'position': idx,
                'actual_timestamp': actual_time,
                'closest_forecast_timestamp': None,
                'time_diff_minutes': None
            })
            continue

        # Find the closest forecast
        min_diff_idx = time_diffs.idxmin()
        min_diff = time_diffs[min_diff_idx]
        closest_forecast_row = df_forecast.loc[min_diff_idx]

        # Check if within 30 minutes tolerance
        if min_diff <= timedelta(minutes=30):
            # Match found
            matched_rows.append({
                # Timestamps
                'actual_timestamp': actual_row['observation_timestamp'],
                'forecast_timestamp': closest_forecast_row['forecast_timestamp'],
                'forecast_scrape_timestamp': closest_forecast_row['scrape_timestamp'],
                'time_diff_minutes': min_diff.total_seconds() / 60,

                # Actual data (prefix with 'actual_')
                'actual_temperature_f': actual_row['temperature_f'],
                'actual_dew_point_f': actual_row['dew_point_f'],
                'actual_humidity_pct': actual_row['humidity_pct'],
                'actual_wind_speed_mph': actual_row['wind_speed_mph'],
                'actual_wind_direction': actual_row['wind_direction'],
                'actual_wind_gust_mph': actual_row['wind_gust_mph'],
                'actual_pressure_in': actual_row['pressure_in'],
                'actual_precip_amount_in': actual_row['precip_amount_in'],
                'actual_condition': actual_row['condition'],
                'actual_water_temp_0_35m_c': actual_row['water_temp_0_35m_c'],
                'actual_water_temp_2m_c': actual_row['water_temp_2m_c'],
                'actual_water_temp_7m_c': actual_row['water_temp_7m_c'],

                # Forecast data (prefix with 'forecast_')
                'forecast_temperature_f': closest_forecast_row['temperature_f'],
                'forecast_feels_like_f': closest_forecast_row['feels_like_f'],
                'forecast_dew_point_f': closest_forecast_row['dew_point_f'],
                'forecast_humidity_pct': closest_forecast_row['humidity_pct'],
                'forecast_wind_speed_mph': closest_forecast_row['wind_speed_mph'],
                'forecast_wind_direction': closest_forecast_row['wind_direction'],
                'forecast_pressure_in': closest_forecast_row['pressure_in'],
                'forecast_precip_chance_pct': closest_forecast_row['precip_chance_pct'],
                'forecast_precip_amount_in': closest_forecast_row['precip_amount_in'],
                'forecast_cloud_cover_pct': closest_forecast_row['cloud_cover_pct'],
                'forecast_condition': closest_forecast_row['condition'],
            })
        else:
            # No matching forecast within 30 minutes
            missing_forecast_positions.append({
                'position': idx,
                'actual_timestamp': actual_time,
                'closest_forecast_timestamp': closest_forecast_row['forecast_timestamp'],
                'time_diff_minutes': min_diff.total_seconds() / 60
            })

    # Check for forecasts that don't have matching actual observations
    matched_forecast_timestamps = [row['forecast_timestamp'] for row in matched_rows]
    for idx, forecast_row in df_forecast.iterrows():
        if forecast_row['forecast_timestamp'] not in matched_forecast_timestamps:
            # Find closest actual observation
            time_diffs = abs(df_actual['observation_timestamp'] - forecast_row['forecast_timestamp'])
            
            # Skip if no actual observations available or all are NaT
            if len(time_diffs) == 0 or time_diffs.isna().all():
                missing_actual_positions.append({
                    'position': idx,
                    'forecast_timestamp': forecast_row['forecast_timestamp'],
                    'closest_actual_timestamp': None,
                    'time_diff_minutes': None
                })
                continue
                
            min_diff_idx = time_diffs.idxmin()
            min_diff = time_diffs[min_diff_idx]
            closest_actual_time = df_actual.loc[min_diff_idx, 'observation_timestamp']

            missing_actual_positions.append({
                'position': idx,
                'forecast_timestamp': forecast_row['forecast_timestamp'],
                'closest_actual_timestamp': closest_actual_time,
                'time_diff_minutes': min_diff.total_seconds() / 60
            })

    # Create aligned dataframe
    df_aligned = pd.DataFrame(matched_rows)

    print(f"✓ Successfully matched {len(matched_rows)} data points")
    print(f"✗ {len(missing_forecast_positions)} actual observations without matching forecasts")
    print(f"✗ {len(missing_actual_positions)} forecasts without matching actual observations")

elif len(df_actual) == 0:
    print("⚠️ No actual observation data available")
    df_aligned = pd.DataFrame()
elif len(df_forecast) == 0:
    print("⚠️ No forecast data available")
    df_aligned = pd.DataFrame()
else:
    df_aligned = pd.DataFrame()

print("="*80)

In [ ]:
# Display the aligned dataframe
print("\n" + "="*80)
print("ALIGNED FORECAST vs ACTUAL DATA")
print("="*80)

if len(df_aligned) > 0:
    print(f"Total aligned data points: {len(df_aligned)}")
    print(f"\nColumn structure:")
    print(f"  - Timestamp columns: actual_timestamp, forecast_timestamp, forecast_scrape_timestamp, time_diff_minutes")
    print(f"  - Actual measurements: actual_temperature_f, actual_humidity_pct, actual_pressure_in, etc.")
    print(f"  - Forecasted values: forecast_temperature_f, forecast_humidity_pct, forecast_pressure_in, etc.")
    print(f"  - Water temperature: actual_water_temp_0_35m_c, actual_water_temp_2m_c, actual_water_temp_7m_c")
    print("="*80)
    print("\nAll aligned data:")
    display(df_aligned)
else:
    print("No aligned data available (no matching timestamps within 30 minutes)")
    print("="*80)

In [ ]:
# Display missing data positions
print("\n" + "="*80)
print("MISSING DATA POSITIONS")
print("="*80)

if len(missing_forecast_positions) > 0:
    print(f"\n{len(missing_forecast_positions)} Actual observations WITHOUT matching forecasts:")
    print("(These actual observations could not be matched to any forecast within 30 minutes)")
    df_missing_forecasts = pd.DataFrame(missing_forecast_positions)
    display(df_missing_forecasts)
else:
    print("\n✓ All actual observations have matching forecasts within 30 minutes")

print("\n" + "-"*80)

if len(missing_actual_positions) > 0:
    print(f"\n{len(missing_actual_positions)} Forecasts WITHOUT matching actual observations:")
    print("(These forecasts could not be matched to any actual observation within 30 minutes)")
    df_missing_actuals = pd.DataFrame(missing_actual_positions)
    display(df_missing_actuals)
else:
    print("\n✓ All forecasts have matching actual observations within 30 minutes")

print("\n" + "="*80)